# 确认python 环境
    # 检查 Python 版本（建议 3.8+）
    python3 --version

    # 安装 venv（如果尚未安装）
    sudo apt update
    sudo apt install python3-venv -y

# 创建虚拟环境
    python3 -m venv venv

# 激活虚拟环境
    source venv/bin/activate

# 升级 pip 并安装依赖
    # 升级 pip（推荐）
    pip install --upgrade pip

# 安装你需要的包，例如：
pip install python-dotenv

# 退出虚拟环境
    deactivate

    # 生成配置信息.
    pip freeze > requirement.txt



要将一个 **Python 虚拟环境（venv）中已通过 `pip` 安装的所有包**（以 `.whl` 形式）完整迁移到另一个 **无法联网的 venv 环境** 中并安装，可以按照以下 **可靠、可复现** 的步骤操作：

---

## ✅ 总体策略

1. **在源机器（有网络）上**：
   - 导出当前 venv 中所有已安装包的列表。
   - 下载这些包及其依赖的 `.whl` 文件（离线安装包集合）。

2. **将这些 `.whl` 文件拷贝到目标机器（无网络）**。

3. **在目标 venv 中离线安装所有包**。

> ⚠️ 注意：不能直接“复制已安装的包文件”，因为 Python 包安装后是解压状态，且路径与环境绑定。正确做法是 **重新安装 `.whl` 文件**。

---

## 🔧 详细步骤

### 第一步：在源机器上导出包列表并下载所有 `.whl` 文件

#### 1. 激活源虚拟环境
```bash
# Linux/macOS
source /path/to/source_venv/bin/activate

# Windows
source_venv\Scripts\activate
```

#### 2. 导出已安装包列表（可选，用于记录）
```bash
pip freeze > requirements.txt
```

#### 3. **下载所有包及其依赖的 `.whl` 文件**
```bash
pip download -d ./offline_wheels -r requirements.txt
```

> ✅ 这会：
> - 自动解析所有依赖；
> - 下载 `.whl`（优先）或 `.tar.gz`（源码包）到 `./offline_wheels` 目录；
> - 保证版本与当前环境一致。

> 💡 如果你不想用 `requirements.txt`，也可以直接下载所有已安装包：
> ```bash
> pip download -d ./offline_wheels $(pip freeze | cut -d'=' -f1)
> ```
> （Linux/macOS 适用；Windows 可用 PowerShell 或先生成列表）

---

### 第二步：将 `offline_wheels` 文件夹复制到目标机器

- 使用 U 盘、内网传输、scp 等方式，把整个 `offline_wheels/` 目录拷贝到目标机器。

---

### 第三步：在目标机器的 venv 中离线安装

#### 1. 创建并激活目标虚拟环境（确保 Python 版本与源一致！）
```bash
python -m venv target_venv
source target_venv/bin/activate   # Linux/macOS
# 或
target_venv\Scripts\activate      # Windows
```

> ⚠️ **重要**：目标机器的 **操作系统 + Python 版本** 必须与源机器兼容，否则 `.whl` 可能无法安装（尤其是含 C 扩展的包如 `numpy`, `cryptography`）。

#### 2. 离线安装所有包
```bash
pip install --no-index --find-links ./offline_wheels -r requirements.txt
```

或者更通用的方式（不依赖 `requirements.txt`）：

```bash
# Linux/macOS
pip install --no-index --find-links ./offline_wheels ./offline_wheels/*.whl

# Windows (PowerShell)
pip install --no-index --find-links ./offline_wheels .\offline_wheels\*.whl
```

> 🔑 参数说明：
> - `--no-index`：禁止访问 PyPI。
> - `--find-links ./offline_wheels`：在该目录中查找依赖。
> - `pip` 会自动按依赖顺序安装。

---

## 🛡️ 注意事项

| 问题 | 解决方案 |
|------|--------|
| **平台不兼容**（如在 Windows 下载的 `.whl` 用在 Linux） | 必须在**目标平台相同**的机器上执行 `pip download` |
| **某些包只有源码（`.tar.gz`）** | 目标机器需安装编译工具（如 `gcc`, `python-dev`），否则会失败 |
| **包依赖冲突** | 使用 `pip download` + `requirements.txt` 可最大程度保持一致性 |
| **大项目依赖复杂** | 建议在 CI 或 Docker 中构建离线包集合，确保可复现 |

---

## ✅ 最佳实践建议

- **始终在目标平台（或相同 OS/Python 版本的机器）上执行 `pip download`**。
- 如果目标机器完全无编译环境，可使用 `--only-binary=:all:` 强制只下载 `.whl`：
  ```bash
  pip download --only-binary=:all: -d ./offline_wheels -r requirements.txt
  ```
  > 若某包没有预编译 wheel，此命令会报错，需提前处理。

---

## 📦 示例完整流程

```bash
# 源机器
source source_venv/bin/activate
pip freeze > requirements.txt
pip download --only-binary=:all: -d ./offline_wheels -r requirements.txt

# 复制 offline_wheels 和 requirements.txt 到目标机器

# 目标机器
python -m venv new_venv
source new_venv/bin/activate
pip install --no-index --find-links ./offline_wheels -r requirements.txt
```

---

如果你告诉我你的操作系统、Python 版本、以及是否有编译环境，我可以给出更具体的命令！